In [1]:
import pandas as pd
import numpy as np
def sessions(sessions):
    p = sessions.groupby('user_id').size().reset_index()
    p = p.rename(columns={0:'total_sessions'})
    
    t = sessions.groupby('user_id').secs_elapsed.mean().reset_index()
    t = t.rename(columns = {'secs_elapsed':'mean'})
    p = pd.merge(p,t, on='user_id',how='left')
    
    t = sessions.groupby('user_id').secs_elapsed.median().reset_index()
    t = t.rename(columns = {'secs_elapsed':'median'})
    p = pd.merge(p,t, on='user_id',how='left')
    
    t = sessions.groupby('user_id').secs_elapsed.std().reset_index()
    t = t.rename(columns = {'secs_elapsed':'std_dev'})
    p = pd.merge(p,t, on='user_id',how='left')
    
    t = sessions.groupby('user_id').secs_elapsed.min().reset_index()
    t = t.rename(columns = {'secs_elapsed':'minimum'})
    p = pd.merge(p,t, on='user_id',how='left')
    
    t = sessions.groupby('user_id').secs_elapsed.max().reset_index()
    t = t.rename(columns = {'secs_elapsed':'maximum'})
    p = pd.merge(p,t, on='user_id',how='left')
    
    for f in ['action','action_type','device_type','action_detail']:
        #,'action_type','device_type','action_detail']:
        t = sessions.groupby(['user_id',f]).size().reset_index()
        t = t.rename(columns={0:'counts'})
        print('hohohohohohohohohoho')
        t = t.pivot(index='user_id',columns = f ,values='counts').reset_index()
        p = pd.merge(p,t,on='user_id',how='left')
    z = sessions
    z['action_action_type_action_detail'] = z['action'] + z['action_type'] + z['action_detail']
    t = z.groupby(['user_id','action_action_type_action_detail']).size().reset_index()
    t = t.pivot(index='user_id',columns='action_action_type_action_detail',values=0).reset_index()
    p=pd.merge(p,t,on='user_id',how='left')
    return p
def f(data):
    data = pd.get_dummies(data, columns=['gender','signup_method','signup_flow','language','affiliate_channel','affiliate_provider','first_affiliate_tracked', 'signup_app','first_device_type','first_browser'], drop_first=False)
    dates = data.date_account_created.apply(lambda x: x.split('-'))
    print(type(dates))
    dates = dates.values
    print(dates)
    dates = np.vstack(dates)
    print(dates)
    year = dates[:,0].astype(int)
    month = dates[:,1].astype(int)
    day = dates[:,2].astype(int)
    print(year)
    print(month)
    print(day)
    print(len(year))
    print(len(month))
    print(len(day))
    print(len(data))
    data['day'] = day
    data['month'] = month
    data['year'] = year
    print(data['year'])
    
    tfa = np.vstack(data.timestamp_first_active.astype(str).apply(lambda x: list(map(int, [x[:4],x[4:6],x[6:8],x[8:10],x[10:12],x[12:14]]))).values)
    data['tfa_year'] = tfa[:,0]
    data['tfa_month'] = tfa[:,1]
    data['tfa_day'] = tfa[:,2]
    
    data = data.drop(['date_account_created'], axis=1)
    data = data.drop(['timestamp_first_active'],axis=1)
    data = data.drop(['date_first_booking'], axis=1)
    av = data.age.values
    data['age'] = np.where(np.logical_or(av<14, av>100), -1, av)
    data.fillna(-1)
    return data

In [2]:
import pandas as pd
sess = pd.read_csv('sessions.csv')
users = pd.read_csv('train_users_2.csv')
test = pd.read_csv('test_users.csv')


In [3]:
users['tt'] = 'train'
test['tt'] = 'test'
tt = pd.concat([users,test],ignore_index=True)

In [4]:
(len(users),len(test),len(tt)-len(users))

(213451, 62096, 62096)

In [5]:
#len(tt.iloc[213451:,:])

In [6]:
test.shape

(62096, 16)

In [7]:
tt.shape

(275547, 17)

In [8]:
tt.head()

,affiliate_channel,affiliate_provider,age,country_destination,date_account_created,date_first_booking,first_affiliate_tracked,first_browser,first_device_type,gender,id,language,signup_app,signup_flow,signup_method,timestamp_first_active,tt
0,direct,direct,NaN,NDF,2010-06-28,NaN,untracked,Chrome,Mac Desktop,-unknown-,gxn3p5htnn,en,Web,0,facebook,20090319043255,train
1,seo,google,38.0,NDF,2011-05-25,NaN,untracked,Chrome,Mac Desktop,MALE,820tgsjxq7,en,Web,0,facebook,20090523174809,train
2,direct,direct,56.0,US,2010-09-28,2010-08-02,untracked,IE,Windows Desktop,FEMALE,4ft3gnwmtx,en,Web,3,basic,20090609231247,train
3,direct,direct,42.0,other,2011-12-05,2012-09-08,untracked,Firefox,Mac Desktop,FEMALE,bjjt8pjhuk,en,Web,0,facebook,20091031060129,train
4,direct,direct,41.0,US,2010-09-14,2010-02-18,untracked,Chrome,Mac Desktop,-unknown-,87mebub9p4,en,Web,0,basic,20091208061105,train


In [9]:
file = f(tt)

<class 'pandas.core.series.Series'>
[['2010', '06', '28'] ['2011', '05', '25'] ['2010', '09', '28'] ...,
 ['2014', '09', '30'] ['2014', '09', '30'] ['2014', '09', '30']]
[['2010' '06' '28']
 ['2011' '05' '25']
 ['2010' '09' '28']
 ..., 
 ['2014' '09' '30']
 ['2014' '09' '30']
 ['2014' '09' '30']]
[2010 2011 2010 ..., 2014 2014 2014]
[6 5 9 ..., 9 9 9]
[28 25 28 ..., 30 30 30]
275547
275547
275547
275547
0         2010
1         2011
2         2010
3         2011
4         2010
5         2010
6         2010
7         2010
8         2010
9         2010
10        2010
11        2010
12        2010
13        2010
14        2010
15        2010
16        2010
17        2010
18        2010
19        2010
20        2010
21        2010
22        2010
23        2010
24        2010
25        2010
26        2010
27        2010
28        2010
29        2010
          ... 
275517    2014
275518    2014
275519    2014
275520    2014
275521    2014
275522    2014
275523    2014
275524    2014
275525  

C:\Program Files\Anaconda3\lib\site-packages\ipykernel\__main__.py:72: RuntimeWarning: invalid value encountered in less
C:\Program Files\Anaconda3\lib\site-packages\ipykernel\__main__.py:72: RuntimeWarning: invalid value encountered in greater


In [10]:
sess = sessions(sess)

hohohohohohohohohoho
hohohohohohohohohoho
hohohohohohohohohoho
hohohohohohohohohoho


In [11]:
data = pd.merge(file,sess,left_on='id',right_on='user_id',how='left')

In [12]:
data.shape

(275547, 1146)

In [13]:
train1 = data.iloc[0:150000,:]
train2 = data.iloc[150000:213451,:]


In [14]:
train = pd.concat([train1,train2])

In [15]:
len(train)

213451

In [16]:
train

,age,country_destination,id,tt,gender_-unknown-,gender_FEMALE,gender_MALE,gender_OTHER,signup_method_basic,signup_method_facebook,...,verify-unknown--unknown-,views-unknown--unknown-,views_campaign-unknown--unknown-,views_campaign_rules-unknown--unknown-,viewviewp3,webcam_upload-unknown--unknown-,weibo_signup_referral_finish-unknown--unknown-,why_host-unknown--unknown-,wishlists-unknown--unknown-,zendesk_login_jwt-unknown--unknown-
0,NaN,NDF,gxn3p5htnn,train,1,0,0,0,0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,38.0,NDF,820tgsjxq7,train,0,0,1,0,0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,56.0,US,4ft3gnwmtx,train,0,1,0,0,1,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,42.0,other,bjjt8pjhuk,train,0,1,0,0,0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,41.0,US,87mebub9p4,train,1,0,0,0,1,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,US,osr2jwljor,train,1,0,0,0,1,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,46.0,US,lsw9q7uk0j,train,0,1,0,0,1,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,47.0,US,0d01nltbrs,train,0,1,0,0,1,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,50.0,US,a1vcnhxeij,train,0,1,0,0,1,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,46.0,US,6uh8zyj2gn,train,1,0,0,0,1,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
test = data[data['tt'] == 'test']

In [18]:
test.shape

(62096, 1146)

In [19]:
train.head()

,age,country_destination,id,tt,gender_-unknown-,gender_FEMALE,gender_MALE,gender_OTHER,signup_method_basic,signup_method_facebook,...,verify-unknown--unknown-,views-unknown--unknown-,views_campaign-unknown--unknown-,views_campaign_rules-unknown--unknown-,viewviewp3,webcam_upload-unknown--unknown-,weibo_signup_referral_finish-unknown--unknown-,why_host-unknown--unknown-,wishlists-unknown--unknown-,zendesk_login_jwt-unknown--unknown-
0,NaN,NDF,gxn3p5htnn,train,1,0,0,0,0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,38.0,NDF,820tgsjxq7,train,0,0,1,0,0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,56.0,US,4ft3gnwmtx,train,0,1,0,0,1,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,42.0,other,bjjt8pjhuk,train,0,1,0,0,0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,41.0,US,87mebub9p4,train,1,0,0,0,1,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
test.head()

,age,country_destination,id,tt,gender_-unknown-,gender_FEMALE,gender_MALE,gender_OTHER,signup_method_basic,signup_method_facebook,...,verify-unknown--unknown-,views-unknown--unknown-,views_campaign-unknown--unknown-,views_campaign_rules-unknown--unknown-,viewviewp3,webcam_upload-unknown--unknown-,weibo_signup_referral_finish-unknown--unknown-,why_host-unknown--unknown-,wishlists-unknown--unknown-,zendesk_login_jwt-unknown--unknown-
213451,35.0,NaN,5uwns89zht,test,0,1,0,0,0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
213452,NaN,NaN,jtl0dijy2j,test,1,0,0,0,1,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
213453,NaN,NaN,xx0ulgorjt,test,1,0,0,0,1,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
213454,NaN,NaN,6c6puo6ix0,test,1,0,0,0,1,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
213455,NaN,NaN,czqhjk3yfe,test,1,0,0,0,1,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
trf = train[train['year'] >= 2014]


In [22]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
'''lef = LabelEncoder()
lef.fit(trf.country_destination.values)
fretar = lef.transform(trf.country_destination.values)'''

'lef = LabelEncoder()\nlef.fit(trf.country_destination.values)\nfretar = lef.transform(trf.country_destination.values)'

In [23]:
leo = LabelEncoder()
leo.fit(train.country_destination.values)
fulltar=leo.transform(train.country_destination.values)
fretar = leo.transform(trf.country_destination.values)

In [24]:
len(fretar),len(fulltar)

(76466, 213451)

In [25]:
test

,age,country_destination,id,tt,gender_-unknown-,gender_FEMALE,gender_MALE,gender_OTHER,signup_method_basic,signup_method_facebook,...,verify-unknown--unknown-,views-unknown--unknown-,views_campaign-unknown--unknown-,views_campaign_rules-unknown--unknown-,viewviewp3,webcam_upload-unknown--unknown-,weibo_signup_referral_finish-unknown--unknown-,why_host-unknown--unknown-,wishlists-unknown--unknown-,zendesk_login_jwt-unknown--unknown-
213451,35.0,NaN,5uwns89zht,test,0,1,0,0,0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
213452,NaN,NaN,jtl0dijy2j,test,1,0,0,0,1,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
213453,NaN,NaN,xx0ulgorjt,test,1,0,0,0,1,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
213454,NaN,NaN,6c6puo6ix0,test,1,0,0,0,1,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
213455,NaN,NaN,czqhjk3yfe,test,1,0,0,0,1,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
213456,28.0,NaN,szx28ujmhf,test,0,1,0,0,1,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
213457,48.0,NaN,guenkfjcbq,test,0,0,1,0,1,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
213458,NaN,NaN,tkpq0mlugk,test,1,0,0,0,1,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
213459,NaN,NaN,3xtgd5p9dn,test,1,0,0,0,1,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
213460,NaN,NaN,md9aj22l5a,test,1,0,0,0,1,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [26]:
(test.shape,trf.shape,train.shape)

((62096, 1146), (76466, 1146), (213451, 1146))

In [27]:
test.head()

,age,country_destination,id,tt,gender_-unknown-,gender_FEMALE,gender_MALE,gender_OTHER,signup_method_basic,signup_method_facebook,...,verify-unknown--unknown-,views-unknown--unknown-,views_campaign-unknown--unknown-,views_campaign_rules-unknown--unknown-,viewviewp3,webcam_upload-unknown--unknown-,weibo_signup_referral_finish-unknown--unknown-,why_host-unknown--unknown-,wishlists-unknown--unknown-,zendesk_login_jwt-unknown--unknown-
213451,35.0,NaN,5uwns89zht,test,0,1,0,0,0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
213452,NaN,NaN,jtl0dijy2j,test,1,0,0,0,1,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
213453,NaN,NaN,xx0ulgorjt,test,1,0,0,0,1,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
213454,NaN,NaN,6c6puo6ix0,test,1,0,0,0,1,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
213455,NaN,NaN,czqhjk3yfe,test,1,0,0,0,1,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [28]:
train.head()

,age,country_destination,id,tt,gender_-unknown-,gender_FEMALE,gender_MALE,gender_OTHER,signup_method_basic,signup_method_facebook,...,verify-unknown--unknown-,views-unknown--unknown-,views_campaign-unknown--unknown-,views_campaign_rules-unknown--unknown-,viewviewp3,webcam_upload-unknown--unknown-,weibo_signup_referral_finish-unknown--unknown-,why_host-unknown--unknown-,wishlists-unknown--unknown-,zendesk_login_jwt-unknown--unknown-
0,NaN,NDF,gxn3p5htnn,train,1,0,0,0,0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,38.0,NDF,820tgsjxq7,train,0,0,1,0,0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,56.0,US,4ft3gnwmtx,train,0,1,0,0,1,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,42.0,other,bjjt8pjhuk,train,0,1,0,0,0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,41.0,US,87mebub9p4,train,1,0,0,0,1,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [29]:
test = test.drop('country_destination',axis=1)
train = train.drop('country_destination',axis=1)
trf = trf.drop('country_destination',axis=1)
(test.shape,train.shape,trf.shape)

((62096, 1145), (213451, 1145), (76466, 1145))

In [30]:
train = train.drop('id',axis=1)
test = test.drop('id',axis=1)
trf = trf.drop('id',axis=1)
trf= trf.drop('tt',axis=1)

In [31]:
train = train.drop('tt',axis=1)
test = test.drop('tt',axis=1)
train =train.fillna(-1)
test=test.fillna(-1)
trf=trf.fillna(-1)
#trf= trf.drop('tt','axis'=1)

In [54]:
import csv
def tocsv(file,name):
    return file.to_csv(name+'.csv', index=False)
tocsv(predictions,'pred1')

AttributeError: 'numpy.ndarray' object has no attribute 'to_csv'

In [35]:
#tocsv(trf,'ffresh')

In [36]:
#tocsv(test,'ftest')

In [37]:
#kk=pd.read_csv('ftest.csv')

In [38]:
'''print(len(kk))
kk.tail()'''

'print(len(kk))\nkk.tail()'

In [39]:
#mm = pd.read_csv('ffresh.csv')

In [40]:
#len(mm)

In [32]:
(train.shape,test.shape,trf.shape)

((213451, 1143), (62096, 1143), (76466, 1143))

In [33]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from xgboost.sklearn import XGBClassifier
def perform_prediction(training, labels, testing, xgb_votes, rf_votes):
    """ Perform prediction using a combination of XGB and RandomForests. """
    predictions = np.zeros((len(testing), len(set(labels))))
    # Predictions using xgboost.
    for i in range(xgb_votes):
        print('XGB vote %d' % i)
        xgb = XGBClassifier(
            max_depth=DEPTH_XGB, learning_rate=LEARNING_XGB,
            n_estimators=ESTIMATORS_XGB, objective='multi:softprob',
            subsample=SUBSAMPLE_XGB, colsample_bytree=COLSAMPLE_XGB,silent=False)
        xgb.fit(training, labels)
        predictions += xgb.predict_proba(testing)
    # Predictions using RandomForestClassifier.
    for i in range(rf_votes):
        print('RandomForest vote %d' % i)
        rand_forest = RandomForestClassifier(
            n_estimators=ESTIMATORS_RF, criterion=CRITERION_RF, n_jobs=JOBS_RF,
            max_depth=DEPTH_RF, min_samples_leaf=MIN_LEAF_RF, bootstrap=True)
        rand_forest.fit(training, labels)
        predictions += rand_forest.predict_proba(testing)
    return predictions

C:\Program Files\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [35]:
predictions = np.zeros((len(test), len(leo.classes_)))
(leo.classes_)

array(['AU', 'CA', 'DE', 'ES', 'FR', 'GB', 'IT', 'NDF', 'NL', 'PT', 'US',
       'other'], dtype=object)

In [36]:
predictions.shape

(62096, 12)

In [37]:
DEPTH_XGB, ESTIMATORS_XGB, LEARNING_XGB, SUBSAMPLE_XGB, COLSAMPLE_XGB = (
    7, 60, 0.2, 0.7, 0.6)                # XGBoost parameters.

ESTIMATORS_RF, CRITERION_RF, DEPTH_RF, MIN_LEAF_RF, JOBS_RF = (
500, 'gini', 20, 8, 30) # RandomForestClassifier parameters.

XGB_ALL_VOTE, RF_ALL_VOTE, XGB_FRESH_VOTE, RF_FRESH_VOTE = (5, 2, 10, 4)

In [38]:
#train = train.iloc[:,:].values

In [39]:
#trf = trf.values

In [40]:
test = test.drop('user_id',axis=1)
train = train.drop('user_id',axis=1)
trf = trf.drop('user_id',axis=1)

In [41]:
#predictions = predictions + perform_prediction(train,fulltar,test,XGB_FRESH_VOTE, RF_FRESH_VOTE)

In [42]:
(train.shape, test.shape, trf.shape)

((213451, 1142), (62096, 1142), (76466, 1142))

In [43]:
#train.user_id

In [44]:
#leo,lef,train,trf, test,fretar,fulltar
predictions += perform_prediction(trf, fretar, test, XGB_FRESH_VOTE, RF_FRESH_VOTE)

XGB vote 0
XGB vote 1
XGB vote 2
XGB vote 3
XGB vote 4
XGB vote 5
XGB vote 6
XGB vote 7
XGB vote 8
XGB vote 9
RandomForest vote 0
RandomForest vote 1
RandomForest vote 2
RandomForest vote 3


In [45]:
predictions

array([[ 0.02346746,  0.02047869,  0.02178847, ...,  0.00556176,
         2.4720409 ,  0.34021897],
       [ 0.0076346 ,  0.00846266,  0.00544644, ...,  0.00474576,
         0.70594791,  0.14068503],
       [ 0.00486506,  0.0509759 ,  0.02045928, ...,  0.01106009,
         0.78325516,  0.24933245],
       ..., 
       [ 0.01537044,  0.05149804,  0.04160888, ...,  0.01187167,
         1.01411548,  0.31458278],
       [ 0.018147  ,  0.03136735,  0.01840666, ...,  0.00543798,
         1.33297027,  0.33746643],
       [ 0.0318171 ,  0.08365634,  0.05527625, ...,  0.02125036,
         2.81416549,  0.47889521]])

In [46]:
#predictions.values.to_csv('preds1.csv',index=False)

In [47]:
predictions += perform_prediction(train, fulltar, test, XGB_ALL_VOTE, RF_ALL_VOTE)

XGB vote 0
XGB vote 1
XGB vote 2
XGB vote 3
XGB vote 4
RandomForest vote 0
RandomForest vote 1


In [58]:
predictions

array([[ 0.03684586,  0.0315593 ,  0.03467507, ...,  0.00868174,
         3.98891308,  0.51987148],
       [ 0.01211785,  0.01294907,  0.00914156, ...,  0.00613535,
         1.05470257,  0.21236394],
       [ 0.00882472,  0.08351397,  0.02692209, ...,  0.01716878,
         1.23725433,  0.35682833],
       ..., 
       [ 0.03383412,  0.07052199,  0.05625465, ...,  0.01640087,
         1.60176456,  0.45734662],
       [ 0.02980122,  0.04739663,  0.02914999, ...,  0.01075548,
         2.01143942,  0.51459598],
       [ 0.05726659,  0.14433455,  0.0845289 , ...,  0.02581958,
         4.85367706,  0.75972469]])

In [59]:
predictions[0]

array([  3.68458551e-02,   3.15593022e-02,   3.46750665e-02,
         1.02008377e-01,   1.73469231e-01,   1.01546742e-01,
         1.77387757e-01,   1.57862339e+01,   3.88071701e-02,
         8.68174233e-03,   3.98891308e+00,   5.19871480e-01])

In [60]:
test

,age,gender_-unknown-,gender_FEMALE,gender_MALE,gender_OTHER,signup_method_basic,signup_method_facebook,signup_method_google,signup_method_weibo,signup_flow_0,...,verify-unknown--unknown-,views-unknown--unknown-,views_campaign-unknown--unknown-,views_campaign_rules-unknown--unknown-,viewviewp3,webcam_upload-unknown--unknown-,weibo_signup_referral_finish-unknown--unknown-,why_host-unknown--unknown-,wishlists-unknown--unknown-,zendesk_login_jwt-unknown--unknown-
213451,35.0,0,1,0,0,0,1,0,0,1,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
213452,-1.0,1,0,0,0,1,0,0,0,1,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
213453,-1.0,1,0,0,0,1,0,0,0,1,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
213454,-1.0,1,0,0,0,1,0,0,0,1,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
213455,-1.0,1,0,0,0,1,0,0,0,1,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
213456,28.0,0,1,0,0,1,0,0,0,1,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
213457,48.0,0,0,1,0,1,0,0,0,0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
213458,-1.0,1,0,0,0,1,0,0,0,1,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
213459,-1.0,1,0,0,0,1,0,0,0,1,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
213460,-1.0,1,0,0,0,1,0,0,0,1,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0


In [52]:
test_df=pd.read_csv('test_users.csv')
len(test_df)

62096

In [50]:
ids, countries = ([], [])
for i in range(len(test_df)):
    idx = test_df.index[i]
    ids += [idx] * 5
    countries += leo.inverse_transform(
        np.argsort(predictions[i])[::-1])[:5].tolist()

    # Save prediction in CSV file.
sub = pd.DataFrame(np.column_stack((ids, countries)), columns=['id', 'country'])

sub.to_csv('1234.csv', index=False)


In [75]:
a = pd.read_csv('SUBMISSION_CSV.csv')

In [54]:
test_df.set_index('id', inplace=True)
id_test = test_df.index.values

In [85]:
id_test

array(['5uwns89zht', 'jtl0dijy2j', 'xx0ulgorjt', ..., 'fa6260ziny',
       '87k0fy4ugm', '9uqfg8txu3'], dtype=object)

In [57]:
######################################################################submission generation#####################################
#Taking the 5 classes with highest probabilities
ids = []  #list of ids
cts = []  #list of countries
for i in range(len(id_test)):
    idx = id_test[i]
    ids += [idx] * 5
    cts += leo.inverse_transform(np.argsort(predictions[i])[::-1])[:5].tolist()

#Generate submission
print("Outputting final results...")
sub = pd.DataFrame(np.column_stack((ids, cts)), columns=['id', 'country'])
sub.to_csv('./submission1234121212.csv', index=False)

Outputting final results...


In [87]:
ids

['5uwns89zht',
 '5uwns89zht',
 '5uwns89zht',
 '5uwns89zht',
 '5uwns89zht',
 'jtl0dijy2j',
 'jtl0dijy2j',
 'jtl0dijy2j',
 'jtl0dijy2j',
 'jtl0dijy2j',
 'xx0ulgorjt',
 'xx0ulgorjt',
 'xx0ulgorjt',
 'xx0ulgorjt',
 'xx0ulgorjt',
 '6c6puo6ix0',
 '6c6puo6ix0',
 '6c6puo6ix0',
 '6c6puo6ix0',
 '6c6puo6ix0',
 'czqhjk3yfe',
 'czqhjk3yfe',
 'czqhjk3yfe',
 'czqhjk3yfe',
 'czqhjk3yfe',
 'szx28ujmhf',
 'szx28ujmhf',
 'szx28ujmhf',
 'szx28ujmhf',
 'szx28ujmhf',
 'guenkfjcbq',
 'guenkfjcbq',
 'guenkfjcbq',
 'guenkfjcbq',
 'guenkfjcbq',
 'tkpq0mlugk',
 'tkpq0mlugk',
 'tkpq0mlugk',
 'tkpq0mlugk',
 'tkpq0mlugk',
 '3xtgd5p9dn',
 '3xtgd5p9dn',
 '3xtgd5p9dn',
 '3xtgd5p9dn',
 '3xtgd5p9dn',
 'md9aj22l5a',
 'md9aj22l5a',
 'md9aj22l5a',
 'md9aj22l5a',
 'md9aj22l5a',
 'gg3eswjxdf',
 'gg3eswjxdf',
 'gg3eswjxdf',
 'gg3eswjxdf',
 'gg3eswjxdf',
 'fyomoivygn',
 'fyomoivygn',
 'fyomoivygn',
 'fyomoivygn',
 'fyomoivygn',
 'iq4kkd5oan',
 'iq4kkd5oan',
 'iq4kkd5oan',
 'iq4kkd5oan',
 'iq4kkd5oan',
 '6k1xls6x5j',
 '6k1xls6x

In [88]:
predictions[0]

array([  3.68458551e-02,   3.15593022e-02,   3.46750665e-02,
         1.02008377e-01,   1.73469231e-01,   1.01546742e-01,
         1.77387757e-01,   1.57862339e+01,   3.88071701e-02,
         8.68174233e-03,   3.98891308e+00,   5.19871480e-01])

In [93]:
predictions = np.vstack(predictions)

In [94]:
predictions

array([[ 0.03684586,  0.0315593 ,  0.03467507, ...,  0.00868174,
         3.98891308,  0.51987148],
       [ 0.01211785,  0.01294907,  0.00914156, ...,  0.00613535,
         1.05470257,  0.21236394],
       [ 0.00882472,  0.08351397,  0.02692209, ...,  0.01716878,
         1.23725433,  0.35682833],
       ..., 
       [ 0.03383412,  0.07052199,  0.05625465, ...,  0.01640087,
         1.60176456,  0.45734662],
       [ 0.02980122,  0.04739663,  0.02914999, ...,  0.01075548,
         2.01143942,  0.51459598],
       [ 0.05726659,  0.14433455,  0.0845289 , ...,  0.02581958,
         4.85367706,  0.75972469]])

In [95]:
p = pd.DataFrame(predictions)

In [97]:
p.to_csv('predictions.csv')

In [51]:
ids, countries = ([], [])
for i in range(len(test_df)):
    idx = test_df.index[i]
    ids += [idx] * 5
    countries += leo.inverse_transform(
        np.argsort(predictions[i])[::-1])[:5].tolist()

    # Save prediction in CSV file.
sub = pd.DataFrame(np.column_stack((ids, countries)), columns=['id', 'country'])

sub.to_csv('SUBMISSION11111111111111111111_CSV.csv', index=False)


In [101]:
len(countries)==len(ids)

True